## Integrantes del equipo

- Myroslava Sánchez Andrade A01730712
- José Antonio Bobadilla García A01734433
- Karen Rugerio Armenta A01733228
- Alejandro Castro Reus A01731065

## English-Spanish translator 

https://www.manythings.org/anki/

https://tatoeba.org/en/

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

#### Cleaining the data

##### From scratch

In [48]:
from google.colab import drive
drive.mount('/content/drive/' , force_remount=True)

Mounted at /content/drive/


In [49]:
import sys
sys.path.append('/content/drive/My Drive/Inteligencia artificial I/NLP/lang')

In [50]:
import random
from lang_processing import *
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [74]:
input_lang, output_lang, pairs = prepareData('eng', 'spa', True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Read 120614 sentence pairs
Trimmed to 7479 sentence pairs
Counting words...
Counted words:
spa 3845
eng 2678


In [52]:
MAX_LENGTH = 10

In [53]:
print(len(pairs))

7479


In [54]:
# rnd_idx = 
print(pairs[11:20])

[['estoy viejo .', 'i am old .'], ['he vuelto .', 'i m back .'], ['estoy de vuelta .', 'i m back .'], ['soy calvo .', 'i m bald .'], ['estoy calvo .', 'i m bald .'], ['estoy calmado .', 'i m calm .'], ['estoy tranquilo .', 'i m cool .'], ['he terminado .', 'i m done .'], ['soy bastante facilon .', 'i m easy .']]


In [55]:
input_lang.word2index

{'tengo': 2,
 'diecinueve': 3,
 '.': 4,
 'estoy': 5,
 'levantado': 6,
 'soy': 7,
 'tom': 8,
 'gordo': 9,
 'gorda': 10,
 'en': 11,
 'forma': 12,
 'afectado': 13,
 'viejo': 14,
 'timido': 15,
 'mojada': 16,
 'he': 17,
 'vuelto': 18,
 'de': 19,
 'vuelta': 20,
 'calvo': 21,
 'calmado': 22,
 'tranquilo': 23,
 'terminado': 24,
 'bastante': 25,
 'facilon': 26,
 'justo': 27,
 'perfectamente': 28,
 '': 29,
 'libre': 30,
 '!': 31,
 'yo': 32,
 'lleno': 33,
 'llena': 34,
 'ya': 35,
 'me': 36,
 'llene': 37,
 'aqui': 38,
 'casa': 39,
 'herido': 40,
 'llego': 41,
 'tarde': 42,
 'vago': 43,
 'perdida': 44,
 'malo': 45,
 'toca': 46,
 'a': 47,
 'mi': 48,
 'pobre': 49,
 'rica': 50,
 'rico': 51,
 'salvo': 52,
 'enferma': 53,
 'delgado': 54,
 'limpio': 55,
 'calentito': 56,
 'debil': 57,
 'listo': 58,
 'estamos': 59,
 'bien': 60,
 'el': 61,
 'es': 62,
 'anciano': 63,
 'dj': 64,
 'veloz': 65,
 'bueno': 66,
 'ocupada': 67,
 'un': 68,
 'hombre': 69,
 'pro': 70,
 'solo': 71,
 'sola': 72,
 'enojado': 73,
 'enoj

### Let us creat the GRU model

In [56]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [57]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        output = self.embedding(x).view(1, 1, -1)
#         output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        # print(output.shape)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [58]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
#         self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, encoder_outputs):
        embedded = self.embedding(x).view(1, 1, -1)
#         embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [59]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [60]:
teacher_forcing_ratio = 0.5


def train_once(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, 
          max_length=MAX_LENGTH, attention=True):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):

            if attention:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
            else:
              decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):

          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


          topv, topi = decoder_output.topk(1)
          decoder_input = topi.squeeze().detach()  # detach from history as input

          loss += criterion(decoder_output, target_tensor[di])
          if decoder_input.item() == EOS_token:
              break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [61]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [62]:
random.choice(pairs)

['vamos a visitar boston .', 'we re going to visit boston .']

In [63]:
def train(encoder, decoder, n_iters, print_every=100, 
               plot_every=100, learning_rate=0.01, attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train_once(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, 
                     criterion, attention=attention)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [64]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [65]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH, attention=True):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)


          if topi.item() == EOS_token:
              decoded_words.append('<EOS>')
              break
          else:
              decoded_words.append(output_lang.index2word[topi.item()])

          decoder_input = topi.squeeze().detach()

        if attention:
          return decoded_words, decoder_attentions[:di + 1]
        else:
          return decoded_words

In [66]:
def evaluateRandomly(encoder, decoder, n=10, attention=True):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        if attention:
          output_words, attentions = evaluate(encoder, decoder, pair[0], attention=attention)
        else:
          output_words = evaluate(encoder, decoder, pair[0], attention=False)

        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [73]:
hidden_size = 256
encoder1 = Encoder(input_lang.n_words, hidden_size).to(device)
decoder1 = AttnDecoder(hidden_size, output_lang.n_words).to(device)
#decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)
train(encoder1, decoder1, 50000, print_every=2500, attention=True)

0m 36s (- 11m 25s) (2500 5%) 3.3612
1m 11s (- 10m 41s) (5000 10%) 2.7936
1m 44s (- 9m 51s) (7500 15%) 2.5027
2m 17s (- 9m 10s) (10000 20%) 2.3235
2m 51s (- 8m 34s) (12500 25%) 2.1354
3m 26s (- 8m 2s) (15000 30%) 1.9423
4m 1s (- 7m 28s) (17500 35%) 1.8252
4m 34s (- 6m 51s) (20000 40%) 1.6735
5m 8s (- 6m 17s) (22500 45%) 1.5920
5m 43s (- 5m 43s) (25000 50%) 1.4669
6m 19s (- 5m 10s) (27500 55%) 1.3084
6m 53s (- 4m 35s) (30000 60%) 1.2594
7m 27s (- 4m 0s) (32500 65%) 1.1838
8m 2s (- 3m 26s) (35000 70%) 1.0776
8m 36s (- 2m 52s) (37500 75%) 1.0255
9m 11s (- 2m 17s) (40000 80%) 0.9971
9m 45s (- 1m 43s) (42500 85%) 0.9085
10m 19s (- 1m 8s) (45000 90%) 0.8139
10m 54s (- 0m 34s) (47500 95%) 0.7813
11m 28s (- 0m 0s) (50000 100%) 0.7088


In [68]:
evaluateRandomly(encoder1, decoder1, attention=True)

>  no eres un espia o si ?
= you aren t a spy are you ?
< i m m to . <EOS>

> ellos estan leyendo su libro .
= they are reading her book .
< i m m to . <EOS>

> el sabe hablar japones .
= he is able to speak japanese .
< i m m to . <EOS>

> estoy planchando mis panuelos .
= i m ironing my handkerchiefs .
< i m m to . <EOS>

> ella es todo para el .
= she is everything to him .
< i m m to . <EOS>

> ya tengo hambre .
= i am hungry .
< i m m to . <EOS>

> ire manana .
= i m going tomorrow .
< i m m to . <EOS>

> el es un excelente neurocirujano .
= he s an excellent brain surgeon .
< i m m to . <EOS>

> ella es curiosa por naturaleza .
= she s inquisitive by nature .
< i m m to . <EOS>

> el es economicamente independiente de sus padres .
= he is economically independent of his parents .
< i m m to . <EOS>

